In [ ]:
#在鐵達尼資料集中，以Titanic_train.csv 中，
#首先將有遺失值的數值刪除，試著將課堂中所學的方法應用上去。

#Q1: 目標變數為 Survived，以 Pclass, Age, SibSp, Parch, Fare,Sex, Embarked 為特徵，
#     試著用今天教授的包裝法，搭配課程所教的 SVC，試著排出其餘特徵的重要性。

In [1]:
# import library
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import math
import statistics
import seaborn as sns
from IPython.display import display

import pingouin as pg
import researchpy   
%matplotlib inline

In [7]:
df_train = pd.read_csv("Titanic_train.csv")
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [5]:
# 会判断哪些”列”存在缺失值
# any：判斷一個tuple或者list是否全為空，0，False。如果全為空，0，False，則返回False；如果不全為空，則返回True。
print(df_train.isnull().any())

PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin           True
Embarked        True
dtype: bool


In [8]:
## 取出資料後，把遺失值刪除
complete_data=df_train[['Survived','Pclass','Age','SibSp','Parch','Fare','Sex','Embarked']].dropna()
display(complete_data)

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked
0,0,3,22.0,1,0,7.2500,male,S
1,1,1,38.0,1,0,71.2833,female,C
2,1,3,26.0,0,0,7.9250,female,S
3,1,1,35.0,1,0,53.1000,female,S
4,0,3,35.0,0,0,8.0500,male,S
...,...,...,...,...,...,...,...,...
885,0,3,39.0,0,5,29.1250,female,Q
886,0,2,27.0,0,0,13.0000,male,S
887,1,1,19.0,0,0,30.0000,female,S
889,1,1,26.0,0,0,30.0000,male,C


In [10]:
from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
#ValueError: could not convert string to float: 'Male'
#離散要轉換成數值
sex_mapping = {
           'male': 1,
           'female': 0}
complete_data['sex1'] = complete_data['Sex'].map(sex_mapping)

embarked_mapping = {
           'S': 1,
           'C': 0,
           'Q': 2}
complete_data['embarked1'] = complete_data['Embarked'].map(embarked_mapping)

x=complete_data[['Pclass','Age','SibSp','Parch','Fare','sex1','embarked1']]
display(x)


,Pclass,Age,SibSp,Parch,Fare,sex1,embarked1
0,3,22.0,1,0,7.2500,1,1
1,1,38.0,1,0,71.2833,0,0
2,3,26.0,0,0,7.9250,0,1
3,1,35.0,1,0,53.1000,0,1
4,3,35.0,0,0,8.0500,1,1
...,...,...,...,...,...,...,...
885,3,39.0,0,5,29.1250,0,2
886,2,27.0,0,0,13.0000,1,1
887,1,19.0,0,0,30.0000,0,1
889,1,26.0,0,0,30.0000,1,0


In [11]:
y=complete_data['Survived']

estimator = SVC(kernel="linear")
selector = RFE(estimator, n_features_to_select=2, step=1)
selector = selector.fit(x, y)
## True= selected feature
print(selector.support_)
#Selected (i.e., estimated best) features are assigned rank 1.
ranking=selector.ranking_
print(ranking)
rfe_feature = x.loc[:,selector.support_].columns.tolist()
print(rfe_feature)

[False False False False False  True  True]
[2 5 4 3 6 1 1]
['sex1', 'embarked1']


In [ ]:
#最能分辨特真的為Sex, Embarked